In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR


In [1]:
# -------------------------------
# 设置中文字体（可选）
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体（黑体）
plt.rcParams['axes.unicode_minus'] = False   # 正常显示负号

In [6]:
#-------------------------------------------------
# 1. 数据处理 —— 数据集 A（原始数据，用于训练模型）
#-------------------------------------------------
file_path_A = "Data/Study1_data.csv" #424个被试
df_A = pd.read_csv(file_path_A)
# 人格问卷数据
cbf_df_A = df_A.copy()
# 音乐评分原始
rating_cols = [
    "您对音频A的感受评价是？", "您对音频B的感受评价是？", "您对音频C的感受评价是？",
    "您对音频D的感受评价是？", "您对音频E的感受评价是？",
    "您对音频F的感受评价是？", "您对音频G的感受评价是？", "您对音频H的感受评价是？",
    "您对音频I的感受评价是？", "您对音频J的感受评价是？",
    "您对音频K的感受评价是？", "您对音频L的感受评价是？", "您对音频M的感受评价是？",
    "您对音频N的感受评价是？", "您对音频O的感受评价是？",
    "您对音频P的感受评价是？", "您对音频Q的感受评价是？", "您对音频R的感受评价是？",
    "您对音频S的感受评价是？", "您对音频T的感受评价是？",
    "您对音频U的感受评价是？", "您对音频V的感受评价是？", "您对音频W的感受评价是？",
    "您对音频X的感受评价是？", "您对音频Y的感受评价是？"
]
ratings_raw_A = df_A[rating_cols].values.astype(np.float32)
ratings_converted_A = ratings_raw_A - 4.0  # 转至 [-3,3]
# ----------------------------------
# 人格问卷反向计分 & 维度计算
# ----------------------------------
cbf_pi_b_columns = { "神经质": [
        "我常感到害怕",
        "有时我觉得自己一无是处",
        "别人一句漫不经心的话，我常会联系在自己身上",
        "在面对压力时，我有种快要崩溃的感觉",
        "我常担忧一些无关紧要的事情",
        "我常常感到内心不踏实",
        "我常担心有什么不好的事情要发生",
        "我很少感到忧郁或沮丧"  # 反向计分
    ],
    "严谨性": [
        "一旦确定了目标，我会坚持努力地实现它",
        "我常常是仔细考虑之后才做出决定",
        "别人认为我是个慎重的人",
        "我喜欢一开头就把事情计划好",
        "我工作或学习很勤奋",
        "我是个倾尽全力做事的人",
        "在工作上，我常只求能应付过去便可",  # 反向计分
        "做事讲究逻辑和条理是我的一个特点"
    ],
    "宜人性": [
        "我觉得大部分人基本上是心怀善意的",
        "我不太关心别人是否受到不公正的待遇",         # 反向计分
        "我时常觉得别人的痛苦与我无关",            # 反向计分
        "我是那种只照顾好自己，不替别人担忧的人",      # 反向计分
        "虽然社会上有些骗子，但我觉得大部分人还是可信的",
        "当别人向我诉说不幸时，我常感到难过",
        "尽管人类社会存在着一些阴暗的东西（如战争、罪恶、欺诈），我仍然相信人性总的来说是善良的",
        "我常为那些遭遇不幸的人感到难过"
    ],
    "开放性": [
        "我头脑中经常充满生动的画面",
        "我是个勇于冒险，突破常规的人",
        "我喜欢冒险",
        "我对许多事情有着很强的好奇心",
        "我身上具有别人没有的冒险精神",
        "我渴望学习一些新东西，即使它们与我的日常生活无关",
        "我的想象力相当丰富",
        "我很愿意也很容易接受那些新事物、新观点、新想法"
    ],
    "外向性": [
        "我对人多的聚会感到乏味",           # 反向计分
        "在热闹的聚会上，我常常表现主动并尽情玩耍",
        "我尽量避免参加人多的聚会和嘈杂的环境",  # 反向计分
        "在一个团体中，我希望处于领导地位",
        "我希望成为领导者而不是被领导者",
        "别人多认为我是一个热情和友好的人",
        "我喜欢参加社交与娱乐聚会",
        "我希望成为领导者而不是被领导者"
    ]}
reverse_items_personality = ["我对人多的聚会感到乏味",
    "我不太关心别人是否受到不公正的待遇",
    "我时常觉得别人的痛苦与我无关",
    "我尽量避免参加人多的聚会和嘈杂的环境",
    "我是那种只照顾好自己，不替别人担忧的人",
    "在工作上，我常只求能应付过去便可",
    "我很少感到忧郁或沮丧"]
# 对反向题目进行 7-原分
for col in reverse_items_personality:
    if col in cbf_df_A.columns:
        cbf_df_A[col] = 7 - cbf_df_A[col]
# 各维度均值 & 归一化
dims = ["神经质", "严谨性", "宜人性", "开放性", "外向性"]
for dim, items in cbf_pi_b_columns.items():
    cbf_df_A[dim] = cbf_df_A[items].mean(axis=1)
for dim in dims:
    cbf_df_A[dim] = (cbf_df_A[dim] - 1) / 5.0




In [7]:
#--------------------------------------------
# 2. 数据处理 —— 数据集 B（外部测试）
#--------------------------------------------
file_path_B = "Data/Study2_data.csv"
df_B = pd.read_csv(file_path_B)
cbf_df_B = df_B.copy()
ratings_raw_B = df_B[rating_cols].values.astype(np.float32)
ratings_converted_B = ratings_raw_B - 4.0
# 反向计分
for col in reverse_items_personality:
    if col in cbf_df_B.columns:
        cbf_df_B[col] = 7 - cbf_df_B[col]
# 维度计算 & 归一化
for dim, items in cbf_pi_b_columns.items():
    cbf_df_B[dim] = cbf_df_B[items].mean(axis=1)
for dim in dims:
    cbf_df_B[dim] = (cbf_df_B[dim] - 1) / 5.0




In [8]:
#------------------------------------------------------------------------
# 自动循环训练：mean, mean_std，full
#------------------------------------------------------------------------
#定义三种方式 - 统计特征函数：mean, mean_std，full
def group_statistics(ratings, stat_type='mean'):
    """
    ratings: ndarray, shape (n_samples, 25)
    stat_type: 'sum', 'mean', or 'mean_std'
    """
    group_indices = [list(range(i, i+5)) for i in range(0, 25, 5)]
    features = []

    for idx in group_indices:
        group = ratings[:, idx]  
        if stat_type == 'mean':
            feat = np.mean(group, axis=1, keepdims=True)
        elif stat_type == 'mean_std':
            mean_ = np.mean(group, axis=1, keepdims=True)
            std_  = np.std(group, axis=1, ddof=0, keepdims=True)
            feat = np.concatenate([mean_, std_], axis=1)
        elif stat_type == 'full':
            feat = np.array([(group == b).sum(axis=1) / group.shape[1] for b in range(-3, 4)]).T
        else:
            raise ValueError(f"Unknown stat_type: {stat_type}")
        features.append(feat)
    return np.concatenate(features, axis=1)

feature_versions = {
     'mean':     {'type': 'mean',     'input_dim': 5},
     'mean_std': {'type': 'mean_std', 'input_dim': 10},
    'full':     {'type': 'full',     'input_dim': 25}
}

In [9]:
#定义逻辑回归类

def sigmoid(x):
    return np.where(x >= 0,
                    1.0 / (1.0 + np.exp(-x)),
                    np.exp(x) / (1.0 + np.exp(x)))


class LogisticRegression:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        num_samples, num_features = X.shape

        # 初始化权重和偏置
        self.weights = np.zeros(num_features)
        self.bias = 0

        # 梯度下降
        for _ in range(self.num_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = sigmoid(linear_model)

            dw = (1 / num_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / num_samples) * np.sum(y_pred - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = sigmoid(linear_model)
        return y_pred


                            

In [ ]:
final_results = {}
for v_name, cfg in feature_versions.items():##输入方式的循环###################################
    
    rf_results, gb_results, svr_results, log_results = {}, {}, {}, {}
    ##数据的准备
    X_A_feat = group_statistics(ratings_converted_A, stat_type=cfg['type']).astype(np.float32)
    X_B_feat = group_statistics(ratings_converted_B, stat_type=cfg['type']).astype(np.float32)

    preds_per_model = {
        "RandomForest": {},
        "GradientBoosting": {},
        "SVR": {},
        "LogisticRegression": {}
    }

    preds_per_model_B = {  # B 测试集预测
        "RandomForest": {},
        "GradientBoosting": {},
        "SVR": {},
        "LogisticRegression": {}
    }
    y_A_matrix = cbf_df_A[dims].values.astype(np.float32)  # shape = (n_subjects, n_dims)
    num_subjects = X_A_feat.shape[0]



    y_B_matrix = cbf_df_B[dims].values.astype(np.float32)  # shape = (n_subjects, n_dims)
    num_subjects_B = X_B_feat.shape[0]


    ##人格维度的循环##########################################################################
    for dim in dims:
        y_A = cbf_df_A[dim].values.astype(np.float32)
        y_B = cbf_df_B[dim].values.astype(np.float32)

        ## 引入10折交叉验证
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        ##折的循环############################################################################
        val_preds_all_rf = [0] * len(X_A_feat)
        val_preds_all_gb = [0] * len(X_A_feat)
        val_preds_all_svr = [0] * len(X_A_feat)
        val_preds_all_log = [0] * len(X_A_feat)
        train_scores_rf, train_scores_gb, train_scores_svr, train_scores_log = [], [], [], []
        for fold, (train_idx, val_idx) in enumerate(kf.split(X_A_feat)):
            X_tr, X_val = X_A_feat[train_idx], X_A_feat[val_idx]
            y_tr, y_val = y_A[train_idx], y_A[val_idx]


            #随机森林回归——————————————————————————————————————————————————————————————————————————————
            rf_model = RandomForestRegressor(
                n_estimators=10,
                max_depth=3,
                random_state=42,
                max_features=0.2, 
                n_jobs=-1,
                min_samples_leaf = 10
                )

            rf_model.fit(X_tr, y_tr)
            pred_train_rf = rf_model.predict(X_tr)
            pred_val_rf = rf_model.predict(X_val)
            
            for idx, i in enumerate(val_idx):
                val_preds_all_rf[i] = pred_val_rf[idx]
            
            # ---------- 每折内评估-------------------------
            train_scores_rf.append([
                np.sqrt(mean_squared_error(y_tr, pred_train_rf)),
                np.corrcoef(y_tr, pred_train_rf)[0, 1]
            ])
           
           
            #梯度提升树模型————————————————————————————————————————————————————————————————————————————————————
            gb_model = GradientBoostingRegressor(
                n_estimators=80,  # 树的数量（100-200之间）
                max_depth=3,    # 或设为适中值，如 5
                max_features=0.1,  # 使用 60% 特征分裂
                random_state=42,
                learning_rate=0.01
            )

            gb_model.fit(X_tr, y_tr)
            pred_val_gb = gb_model.predict(X_val)
            pred_train_gb = gb_model.predict(X_tr)

            for idx, i in enumerate(val_idx):
                val_preds_all_gb[i] = pred_val_gb[idx]
            
            # ---------- 每折内评估-------------------------
            train_preds_gb = rf_model.predict(X_tr)
            rmse_train_gb = np.sqrt(mean_squared_error(y_tr, train_preds_gb))
        
            corr_train_gb = np.corrcoef(y_tr, train_preds_gb)[0, 1]
           
            train_scores_gb.append([
                np.sqrt(mean_squared_error(y_tr, pred_train_gb)),
                np.corrcoef(y_tr, pred_train_gb)[0, 1]
            ])

            #支持向量回归——————————————————————————————————————————————————————————————————————————————————————
            svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)  
            svr_model.fit(X_tr, y_tr)
            
            pred_val_svr = svr_model.predict(X_val)
            pred_train_svr = svr_model.predict(X_tr)

            for idx, i in enumerate(val_idx):
                val_preds_all_svr[i] = pred_val_svr[idx]
            
            # ---------- 每折内评估-------------------------
            train_scores_svr.append([
                np.sqrt(mean_squared_error(y_tr, pred_train_svr)),
                np.corrcoef(y_tr, pred_train_svr)[0, 1]
            ])



            #逻辑回归——————————————————————————————————————————————————————————————————————————————————————————
            log_model = LogisticRegression()
            
            log_model.fit(X_tr, y_tr) 

            pred_val_log = log_model.predict(X_val)
            pred_train_log = log_model.predict(X_tr)

            for idx, i in enumerate(val_idx):
                val_preds_all_log[i] = pred_val_log[idx]
                
            # ---------- 每折内评估-------------------------
            train_scores_log.append([
                np.sqrt(mean_squared_error(y_tr, pred_train_log)),
                np.corrcoef(y_tr, pred_train_log)[0, 1]
            ])

        #评估——————————————————————————————————————————————————————————————————————————————————————————————       
        # ===================== 求训练集指标均值 =====================
        train_rmse_rf, train_corr_rf = np.mean(train_scores_rf, axis=0)
        train_rmse_gb, train_corr_gb = np.mean(train_scores_gb, axis=0)
        train_rmse_svr, train_corr_svr = np.mean(train_scores_svr, axis=0)
        train_rmse_log, train_corr_log = np.mean(train_scores_log, axis=0)
       
        #rf
        rmse_v_rf = np.sqrt(mean_squared_error(y_A, val_preds_all_rf))
        corr_v_rf = np.corrcoef(y_A, val_preds_all_rf)[0, 1]

        #GBDT
        rmse_v_gb = np.sqrt(mean_squared_error(y_A, val_preds_all_gb))
        corr_v_gb = np.corrcoef(y_A, val_preds_all_gb)[0, 1]

        #SVR
        rmse_v_svr = np.sqrt(mean_squared_error(y_A, val_preds_all_svr))
        corr_v_svr = np.corrcoef(y_A, val_preds_all_svr)[0, 1]

        #logistic
        rmse_v_log = np.sqrt(mean_squared_error(y_A, val_preds_all_log))
        corr_v_log = np.corrcoef(y_A, val_preds_all_log)[0, 1]

        # 训练A全集模型用于B测试（保证和原始一致，10折不影响B评估）#######################################################
        #随机森林模型——————————————————————————————————————————————————————————————————————————————————————————

        rf_model.fit(X_tr, y_tr)
            
        pred_B_rf = rf_model.predict(X_B_feat)

        # 评估
        mse_B_rf = mean_squared_error(y_B, pred_B_rf)
        rmse_B_rf= np.sqrt(mse_B_rf)
        corr_B_rf= np.corrcoef(y_B, pred_B_rf)[0,1]

        #梯度提升树————————————————————————————————————————————————————————————————————————————————————————————

        gb_model.fit(X_tr, y_tr)
        
        pred_B_gb = gb_model.predict(X_B_feat)

        # 评估
        mse_B_gb = mean_squared_error(y_B, pred_B_gb)
        rmse_B_gb= np.sqrt(mse_B_gb)
        corr_B_gb= np.corrcoef(y_B, pred_B_gb)[0,1]
        
        #支持向量回归——————————————————————————————————————————————————————————————————————————————————————————
        svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1) 
        svr_model.fit(X_A_feat, y_A)
        pred_B_svr = svr_model.predict(X_B_feat)

        #评估
        rmse_B_svr = np.sqrt(mean_squared_error(y_B, pred_B_svr))
        corr_B_svr = np.corrcoef(y_B, pred_B_svr)[0, 1]

        #逻辑回归————————————————————————————————————————————————————————————————————————————————————————————————
        log_model = LogisticRegression(    )
        log_model.fit(X_A_feat, y_A)
        pred_B_log = log_model.predict(X_B_feat)
        
        #评估
        mse_B_log = mean_squared_error(y_B, pred_B_log)
        rmse_B_log = np.sqrt(mse_B_log)
        corr_B_log = np.corrcoef(y_B, pred_B_log)[0, 1]


        #结果记录####################################################################################################
        rf_results[dim] = pd.Series({
            'A_Train_RMSE': train_rmse_rf, 'A_Train_Corr': train_corr_rf,
            'A_Val_RMSE': rmse_v_rf, 'A_Val_Corr': corr_v_rf,
            'B_Test_RMSE': rmse_B_rf, 'B_Test_Corr': corr_B_rf
        })

        gb_results[dim] = pd.Series({
            'A_Train_RMSE': train_rmse_gb, 'A_Train_Corr': train_corr_gb,
            'A_Val_RMSE': rmse_v_gb,  'A_Val_Corr': corr_v_gb,
            'B_Test_RMSE': rmse_B_gb,  'B_Test_Corr': corr_B_gb
        })

        svr_results[dim] = pd.Series({
            'A_Train_RMSE': train_rmse_svr,  'A_Train_Corr': train_corr_svr,
            'A_Val_RMSE': rmse_v_svr,  'A_Val_Corr': corr_v_svr,
            'B_Test_RMSE': rmse_B_svr, 'B_Test_Corr': corr_B_svr
        })

        log_results[dim] = pd.Series({
            'A_Train_RMSE': train_rmse_log, 'A_Train_Corr': train_corr_log,
            'A_Val_RMSE': rmse_v_log, 'A_Val_Corr': corr_v_log,
            'B_Test_RMSE': rmse_B_log,'B_Test_Corr': corr_B_log
        })


        preds_per_model["RandomForest"][dim] = np.array(val_preds_all_rf)
        preds_per_model["GradientBoosting"][dim] = np.array(val_preds_all_gb)
        preds_per_model["SVR"][dim] = np.array(val_preds_all_svr)
        preds_per_model["LogisticRegression"][dim] = np.array(val_preds_all_log)

        preds_per_model_B["RandomForest"][dim] = pred_B_rf
        preds_per_model_B["GradientBoosting"][dim] = pred_B_gb
        preds_per_model_B["SVR"][dim] = pred_B_svr
        preds_per_model_B["LogisticRegression"][dim] = pred_B_log




    records = []
    for subj_idx in range(num_subjects):
        subj_name = f"subj_{subj_idx+1}"
        for dim_idx, dim in enumerate(dims):
            rf_err = abs(preds_per_model["RandomForest"][dim][subj_idx] - y_A_matrix[subj_idx, dim_idx])
            gb_err = abs(preds_per_model["GradientBoosting"][dim][subj_idx] - y_A_matrix[subj_idx, dim_idx])
            svr_err = abs(preds_per_model["SVR"][dim][subj_idx] - y_A_matrix[subj_idx, dim_idx])
            log_err = abs(preds_per_model["LogisticRegression"][dim][subj_idx] - y_A_matrix[subj_idx, dim_idx])
            records.append([subj_name, dim, rf_err, gb_err, svr_err, log_err])

    df_model_errors = pd.DataFrame(
        records,
        columns=["subject_id", "trait",
                 f"{v_name}_RandomForest",
                 f"{v_name}_GradientBoosting",
                 f"{v_name}_SVR",
                 f"{v_name}_LogisticRegression"]
    )

    wide_path = r"Detailed_Output/A_wide_abs_errors_model1.csv"
    wide_df = pd.read_csv(wide_path, encoding="utf-8-sig")
    merged = wide_df.merge(df_model_errors, on=["subject_id", "trait"], how="left")

    save_path = r"Detailed_Output\A_wide_abs_errors_final.csv"
    merged.to_csv(save_path, index=False, encoding="utf-8-sig")

#改动开始
    # ---------------- B 宽表 ----------------
    records_B = []
    for subj_idx in range(num_subjects_B):
        subj_name = f"subj_{subj_idx+1}"
        for dim_idx, dim in enumerate(dims):
            rf_err = abs(preds_per_model_B["RandomForest"][dim][subj_idx] - y_B_matrix[subj_idx, dim_idx])
            gb_err = abs(preds_per_model_B["GradientBoosting"][dim][subj_idx] - y_B_matrix[subj_idx, dim_idx])
            svr_err = abs(preds_per_model_B["SVR"][dim][subj_idx] - y_B_matrix[subj_idx, dim_idx])
            log_err = abs(preds_per_model_B["LogisticRegression"][dim][subj_idx] - y_B_matrix[subj_idx, dim_idx])
            records_B.append([subj_name, dim, rf_err, gb_err, svr_err, log_err])
    df_model_errors_B = pd.DataFrame(records_B,
        columns=["subject_id","trait",
                 f"{v_name}_RandomForest",f"{v_name}_GradientBoosting",
                 f"{v_name}_SVR",f"{v_name}_LogisticRegression"])
    wide_path_B = r"Detailed_Output\B_wide_abs_errors_final.csv"
    if os.path.exists(wide_path_B):
        wide_df_B = pd.read_csv(wide_path_B, encoding="utf-8-sig")
        merged_B = wide_df_B.merge(df_model_errors_B, on=["subject_id", "trait"], how="left")
    else:
        merged_B = df_model_errors_B
    merged_B.to_csv(wide_path_B, index=False, encoding="utf-8-sig")



    final_results[v_name] = {
        "RandomForest": pd.DataFrame(rf_results).T,
        "GradientBoosting": pd.DataFrame(gb_results).T,
        "SVR": pd.DataFrame(svr_results).T,
        "LogisticRegression": pd.DataFrame(log_results).T
    }


In [12]:
output_path = r"Main_Result/Machine_Learning.txt"
print(final_results)
with open(output_path, "w", encoding="utf-8") as f:
    for v_name, models_result in final_results.items():
        f.write(f"\n================ 特征版本: {v_name} ================\n\n")
        for model_name, df in models_result.items():
            f.write(f"--- 模型: {model_name} ---\n")
            f.write(df.round(4).to_string())
            f.write("\n\n")

{'mean': {'RandomForest':      A_Train_RMSE  A_Train_Corr  A_Val_RMSE  A_Val_Corr  B_Test_RMSE  \
神经质      0.178616      0.421967    0.184364    0.318193     0.209520   
严谨性      0.099468      0.393154    0.103465    0.239436     0.110432   
宜人性      0.111177      0.405119    0.116479    0.253538     0.133345   
开放性      0.133575      0.451312    0.139699    0.325359     0.166164   
外向性      0.180080      0.472981    0.186713    0.369907     0.211085   

     B_Test_Corr  
神经质     0.301794  
严谨性     0.426626  
宜人性     0.346291  
开放性     0.385376  
外向性     0.363727  , 'GradientBoosting':      A_Train_RMSE  A_Train_Corr  A_Val_RMSE  A_Val_Corr  B_Test_RMSE  \
神经质      0.179414      0.527049    0.186271    0.330094     0.207013   
严谨性      0.099526      0.533767    0.104354    0.221706     0.113737   
宜人性      0.111779      0.511210    0.117226    0.246791     0.134490   
开放性      0.134373      0.535066    0.140022    0.358331     0.168500   
外向性      0.180859      0.566561    0.188182   